In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import metrics
import catboost as cb
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from mlxtend.preprocessing import minmax_scaling
# for Box-Cox Transformation
from scipy import stats
import xgboost as xgb
from sklearn.metrics import roc_auc_score

from sklearn import preprocessing
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import warnings 
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('/kaggle/input/financialresilience1234/Train.csv')
submission = pd.read_csv('/kaggle/input/financialresilience1234/SampleSubmission.csv')
test = pd.read_csv('/kaggle/input/financialresilience1234/Test.csv')
variable = pd.read_csv('/kaggle/input/financialresilience1234/VariableDefinitions.csv')

In [6]:
def preprocess(train,test):
    #catogiry
    
    cat_columns = train.select_dtypes(include=['object'])
    cat_columns = cat_columns.fillna('East Asia').replace({'Taiwan, China':'Taiwan'})
    
    train.loc[:,['country','region']]  = cat_columns[['country','region']]
    test.loc[:,['country','region']]  = cat_columns[['country','region']]
    
    #normalizaiont population
    original_data = train.population
    normalized_data = stats.boxcox(original_data)
    df.population = normalized_data[0]
    #scaled 
#     scaled_data = minmax_scaling(df.population, columns=[0])
#     df.population = normalized_data[0]

# Fill in the median below: imputation
    smimpute = SimpleImputer(strategy='median')
    numeric_data = train.select_dtypes(exclude=['object'])
    imputed_with_median = pd.DataFrame(smimpute.fit_transform(numeric_data),columns=numeric_data.columns)
    train.loc[:,numeric_data.columns] = imputed_with_median

    
    numeric_datat = test.select_dtypes(exclude=['object'])
    imputed_with_mediant = pd.DataFrame(smimpute.fit_transform(numeric_datat),columns=numeric_datat.columns)
    test.loc[:,numeric_datat.columns] = imputed_with_mediant
    
    #Label Incode
    le = preprocessing.LabelEncoder()
    le.fit(train.country)
    train.country = le.transform(train.country)
    le.fit(test.country)
    test.country = le.transform(test.country)
    
    #One Hot Encoding
    # Apply one-hot encoder to each column with categorical data
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[['region']]))
    OH_cols_train.index = train.index
    train = train.drop(['region'],axis=1)
    train = pd.concat([train, OH_cols_train], axis=1)
    
    OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(test[['region']]))
    OH_cols_test.index = test.index
    test = test.drop(['region'],axis=1)
    test = pd.concat([test, OH_cols_test], axis=1)
    
    
    
    return train,test

In [7]:
train,test = preprocess(df.copy(),test.copy())

In [8]:
X = train.drop(['target','ID'],axis=1)
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# GridSearch CV

In [25]:
SEED = 42

In [26]:
cross_valid_scores = {}

In [27]:
%%time
parameters = {
    'max_depth': [3, 5, 7, 9], 
    'n_estimators': [100,200,300,400],
    'learning_rate': [0.01, 0.05, 0.1]
}

model_xgb = xgb.XGBClassifier(
    random_state=SEED,
    tree_method='gpu_hist'
)

model_xgb = GridSearchCV(
    model_xgb, 
    parameters, 
    cv=5,
    scoring='roc_auc'
)

model_xgb.fit(X_train, y_train)

CPU times: user 3min 55s, sys: 2.29 s, total: 3min 57s
Wall time: 3min 10s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                  

In [28]:
print('-----')
print(f'Best parameters {model_xgb.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_xgb.best_score_:.3f}'
)
cross_valid_scores['xgboost'] = model_xgb.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}
Mean cross-validated accuracy score of the best_estimator: 0.795
-----


In [35]:
%%time
parameters = {
    "n_estimators": [5, 10, 15, 20, 25, 50, 75, 100], 
    "learning_rate": [0.001, 0.01, 0.1, 1.],
}

model_adaboost = AdaBoostClassifier(
    random_state=42
)

model_adaboost = GridSearchCV(
    model_adaboost, 
    parameters, 
    cv=5,
    scoring='roc_auc',
)

model_adaboost.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_adaboost.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_adaboost.best_score_:.3f}'
)
cross_valid_scores['ada_boost'] = model_adaboost.best_score_
print('-----')

-----
Best parameters {'learning_rate': 1.0, 'n_estimators': 100}
Mean cross-validated accuracy score of the best_estimator: 0.776
-----
CPU times: user 5min 32s, sys: 98 ms, total: 5min 32s
Wall time: 5min 33s


In [30]:
%%time
parameters = {
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [7, 15, 31],
}

model_lgbm = lgbm.LGBMClassifier(
    random_state=42,
    device =  "gpu"
)

model_lgbm = GridSearchCV(
    model_lgbm, 
    parameters, 
    cv=5,
    scoring='roc_auc',
)

model_lgbm.fit(
    X_train, 
    y_train
)

print('-----')
print(f'Best parameters {model_lgbm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_lgbm.best_score_:.3f}'
)
cross_valid_scores['lightgbm'] = model_lgbm.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 31}
Mean cross-validated accuracy score of the best_estimator: 0.795
-----
CPU times: user 4min 48s, sys: 37.2 s, total: 5min 25s
Wall time: 3min 22s


In [ ]:
# %%time
# parameters = {
#     # commet this bellow
# #     'iterations': [5, 10, 15, 20, 25, 50, 100],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'depth': [3, 5, 7, 9, 11, 13],
# }

# model_catboost = cb.CatBoostClassifier(
#     # uncomment this bello
#     iterations = 10,
#     verbose=False,
#     task_type="GPU",
# )

# model_catboost = GridSearchCV(
#     model_catboost, 
#     parameters, 
#     cv=5,
#     scoring='roc_auc',
# )

# model_catboost.fit(X_train, y_train)

# print('-----')
# print(f'Best parameters {model_catboost.best_params_}')
# print(
#     f'Mean cross-validated accuracy score of the best_estimator: ' + 
#     f'{model_catboost.best_score_:.3f}'
# )
# cross_valid_scores['catboost'] = model_catboost.best_score_
# print('-----')

In [ ]:
# %%time
# parameters = {
#     "C": [0.001, 0.01, 0.1, 1.],
#     "penalty": ["l1", "l2"],
#     "class_weight":["balanced",None],
#     "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
# }

# model_logistic_regression = LogisticRegression(
#     random_state=42
# )

# model_logistic_regression = GridSearchCV(
#     model_logistic_regression, 
#     parameters, 
#     cv=5,
#     scoring='roc_auc',
# )

# model_logistic_regression.fit(X_train, y_train)

# print('-----')
# print(f'Best parameters {model_logistic_regression.best_params_}')
# print(
#     f'Mean cross-validated accuracy score of the best_estimator: ' + 
#     f'{model_logistic_regression.best_score_:.3f}'
# )
# cross_valid_scores['logistic_regression'] = model_logistic_regression.best_score_
# print('-----')

In [47]:
%%time
parameters = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

model_k_neighbors = KNeighborsClassifier()

model_k_neighbors = GridSearchCV(
    model_k_neighbors, 
    parameters, 
    cv=5,
    scoring ='roc_auc'
)

model_k_neighbors.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_k_neighbors.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_k_neighbors.best_score_:.3f}'
)
cross_valid_scores['k_neighbors'] = model_k_neighbors.best_score_
print('-----')

KeyboardInterrupt: 

In [31]:
# %%time
# parameters = {
#     "n_estimators": [5, 10, 15, 20, 25], 
#     "max_depth": [3, 5, 7, 9, 11, 13],
#     "class_weight":["balanced",None]
# }

# model_random_forest = RandomForestClassifier(
#     random_state=42
# )

# model_random_forest = GridSearchCV(
#     model_random_forest, 
#     parameters, 
#     cv=5,
#     scoring='roc_auc',
# )

# model_random_forest.fit(X_train, y_train)

# print('-----')
# print(f'Best parameters {model_random_forest.best_params_}')
# print(
#     f'Mean cross-validated accuracy score of the best_estimator: '+ \
#     f'{model_random_forest.best_score_:.3f}'
# )
# cross_valid_scores['random_forest'] = model_random_forest.best_score_
# print('-----')

In [36]:
pd.DataFrame(cross_valid_scores, index=['cross_valid_score']).T

,cross_valid_score
xgboost,0.794812
lightgbm,0.795164
ada_boost,0.775580


In [44]:
def create_submission(model, test, model_name):
    y_pred_test = model.predict_proba(test.drop(['ID'],axis=1))[:, 1]
    sub = pd.DataFrame(
        {
            'ID': submission.ID, 
            'target': y_pred_test,
        }
        
    )
    sub.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test

In [46]:
# test_pred_random_forest = create_submission(
#     model_random_forest, test, "random_forest"
# )
test_pred_adaboost = create_submission(
    model_adaboost, test, "adaboost"
)
test_pred_xgboost = create_submission(
    model_xgb,  test, "xgboost"
)
test_pred_lightgbm = create_submission(
    model_lgbm, test, "lightgbm"
)
# test_pred_catboost = create_submission(
#     model_catboost, test, "catboost"
# )
# test_pred_logistic_regression = create_submission(
#     model_logistic_regression, test, "logistic_regression"
# )

# test_pred_k_neighbors = create_submission(
#     model_k_neighbors, test, "k_neighbors"
# )

In [48]:
test_pred_lightgbm

array([0.8891577 , 0.37711372, 0.70791568, ..., 0.57724756, 0.45723167,
       0.32063326])